# Read in dust map and overplot X-ray sources in 3D

Using a dust map generated by GenerateDustCubes.py, we plot that in 3D alongside the 3D positions of the X-ray sources.

First we have to read in the list of X-ray sources and figure out their coordinates.  Then we will plot.

In [369]:
import numpy as np
import ipyvolume as ipv
from genfire.fileio import readMRC
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
#from astroquery.esa.xmm_newton import XMMNewton
from astroquery.simbad import Simbad
Simbad.add_votable_fields('distance')
from ipywidgets import IntProgress

# Function to convert our cartesian coordinates to spherical coordinates (i.e. galactic coordinates).
#@njit
def Spherical2Cartesian(r, phi, theta):
    phi = np.deg2rad(phi)
    theta = np.deg2rad(theta+90)
    x = r * np.cos(phi) * np.sin(theta)
    y = r * np.sin(phi) * np.sin(theta)
    z = r * np.cos(theta)
    return x, y, z


drho1kpc = readMRC('drho 1kpc.mrc')
drho10kpc = readMRC('drho 10kpc.mrc')

In [76]:
# Observations.csv just has a list of bright sources mined from the Chandra data.  These are probably the top N interesting X-ray sources in the sky.  So let's look at them.
Sources = pd.read_csv('Observations.csv', skiprows=6, delimiter='|')
display(Sources.describe())
display(Sources.head())

,ObsID,TStart,TStop,RA,Dec
count,1275.000000,1.275000e+03,1.275000e+03,1275.000000,1275.000000
mean,9802.858039,3.297251e+08,3.297831e+08,193.587378,-9.870585
std,6890.041809,1.801658e+08,1.801528e+08,90.868515,41.936426
min,3.000000,5.221400e+07,5.222980e+07,6.727649,-79.361416
25%,3672.000000,1.481684e+08,1.482524e+08,109.682285,-42.371310
50%,9939.000000,3.422915e+08,3.423131e+08,196.385246,-17.581536
75%,15172.500000,4.820186e+08,4.820978e+08,266.567625,31.311291
max,62877.000000,6.653697e+08,6.653993e+08,358.761780,79.770103


,ObsID,Object,TStart,TStop,RA,Dec
0,13089,Capella,4.075548e+08,4.075864e+08,79.176373,45.994974
1,20099,GX340+0,6.139789e+08,6.140465e+08,251.454198,-45.609426
2,6628,GX 349+2,2.684048e+08,2.684187e+08,256.434140,-36.424890
3,13841,Sgr A*,4.589465e+08,4.589932e+08,266.419918,-29.008846
4,6617,XTE J1817-330,2.587939e+08,2.588417e+08,274.433491,-33.020187


In [272]:
# Query Simbad to get the entries for all the X-ray objects.
SimbadSources = Simbad.query_objects(Sources['Object'])

/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/conda37/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning:


/home/zack/anaconda3/envs/co

In [320]:
NumObservationsToCalculate = -1

def GetFullCoordsForObject(s):
#     # It is possible that the object name refers to an object that isn't in Simbad.  Nothing we can do with that.
#     if s is None:
#         return np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
    
    # Get the coordinates of this object
    try:
        cs = SkyCoord.from_name(s['MAIN_ID'])
    except:
        # If any trouble finding this source then move on.
        return np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
    
    # If this entry doesn't have a distance recorded by Simbad, then we have no position to report
    if np.isnan(s['Distance_distance']):
        return np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
    
    # A distance is measured so check the units.
    if s['Distance_unit'] == 'pc':
        distunit = u.pc
    elif s['Distance_unit'] == 'kpc':
        distunit = u.kpc
    elif s['Distance_unit'] == 'Mpc':
        distunit = u.Mpc
    else:
        raise TypeError('Unknown distance unit: %s' % s['Distance_unit'])
        
    # And finally convert to the galactic and xyz coordinates we are using for the map.    
    c = SkyCoord(cs.galactic.l.degree*u.deg, cs.galactic.b.degree*u.deg, s['Distance_distance']*distunit, frame='galactic')
    x,y,z = Spherical2Cartesian(r=c.distance.pc, phi=c.l.deg, theta=c.b.deg)
    return c.l.deg, c.b.deg, c.distance.pc, x, y, z

l = [] # Galactic longitude of object in deg.
b = [] # Gal lat deg
d = [] # Distance in pc
x = [] # xyz, distance in pc
y = []
z = []
#for i, r in Sources.head(NumObservationsToCalculate).iterrows():
ProgBar = IntProgress(min=0, max=SimbadSources.to_pandas().shape[0])
display(ProgBar)
for i, r in SimbadSources.to_pandas().head(NumObservationsToCalculate).iterrows():
    ProgBar.value = i
    print(r['MAIN_ID'].decode())
    c = GetFullCoordsForObject(r)
    l.append(c[0]), b.append(c[1]), d.append(c[2]), x.append(c[3]), y.append(c[4]), z.append(c[5])
    
ScatterSources = SimbadSources.to_pandas().head(NumObservationsToCalculate).copy()
ScatterSources['l'] = l
ScatterSources['b'] = b
ScatterSources['dist'] = d
ScatterSources['x'] = x
ScatterSources['y'] = y
ScatterSources['z'] = z
ScatterSources

IntProgress(value=0, max=1158)

* alf Aur
4U 1642-45
V* V1101 Sco
NAME Sgr A*
[KRL2007b] 312
CD-57  3057
NAME Centaurus A
V* FK Com
ICRF J184208.9+794617
HD 226868
HD 193793
V* V404 Cyg
NAME Sgr A*
LIN 198
GRB 021004
* tet Car
X LMC X-3
X LMC X-3
QSO B2155-304
QSO B2149-306
V* KZ TrA
SN 1987A
ESO 445-50
* mu. Vel
* eta Car
V* V1055 Ori
V* YY Dra
3C 273
V* V1101 Sco
V* V1101 Sco
ESO 445-50
SN 1996cr
QSO B2155-304
Mrk 1095
V* V1101 Sco
V* BP Cru
SN 1987A
NAME Sgr A*
V* V818 Sco
* alf Aur
HD  22468
* ksi Per
Ton  951
V* V645 Cen
NAME Sgr A*
3C 454.3
PB  3894
* zet Pup
NAME XTE J17464-3213
4C 71.07
[KRL2007b] 222
V* CM Tau
V* V1033 Sco
ESO 323-77
NAME XTE J17464-3213
Granat 1915+105
NGC   863
MAXI J1535-571
* zet Pup
[KRL2007b] 222
V* BR Cir
V* V2293 Oph
4U 1850-03
NAME Sgr A*
4U 1758-25
Granat 1915+105
* alf Aur
V* V1408 Aql
V* SU UMa
[KRL2007b] 222
IGR J17062-6143
HD 119682
ICRF J183503.3+324146
V* GK Per
* alf Aur
V* V821 Ara
V* SU Aur
Mrk  290
* tau CMa
V* V1405 Aql
SN 1987A
4U 1758-25
NAME Sgr A*
SN 1996cr
HD 191612

2MASX J14260850+4247494
QSO B1830-211
HD 193793
HD 206267
*   9 Sgr
HD  93250
V* T Cha
[KRL2007b] 312
Mrk  509
Mrk  841
NGC  4051
V* NY Lup
HD 119682
NAME Sgr A*
NAME XTE J17464-3213
EM* AS  431
NGC  4051
NGC  2110
NGC  3516
* sig Gem
LEDA   68751
2XMM J180112.4-254436
NAME Hya A
SK  160
NGC  4051
EM* AS  431
V* CP Pup
V* V1521 Cyg
V* V1521 Cyg
4U 1642-45
V* GR Mus
NAME MR 2251-178
HD  42054
NGC  4051
NAME Hya A
V* T Tau
4U 1823-00
NGC  4051
SN 1987A
3C  84
NGC  4051
ICRF J184208.9+794617
* tau Sco
* eta Car
M  87
ESO 198-24
* del Ori
V* V2606 Oph
V* RY Lup
M  77
QSO B1028+511
NGC  7314
CPD-63  2495
HD 226868
Mrk  876
NGC  4253
NAME Slow Burster
SS 433
NGC  3783
QSO B2155-304
V* V1223 Sgr
ESO 434-40
X LMC X-1
HD  47129
Mrk    3
NAME OAO 1657-41
X LMC X-2
NGC  7582
HD 155555
V* V691 CrA
* zet Ori
V* U Gem
V* KZ TrA
NGC  3516
NGC  7469
NGC  2110
NGC  2110
2MASX J14062191+2223462
NAME Rapid Burster
V* IM Peg
* zet Pup
IC  348
X LMC X-4
NAME TER 5 X-1
V* BO Mic
NAME SNR 1987A
QSO B1725-142

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_merr_2,Distance_perr_2,Distance_method_2,Distance_bibcode_2,l,b,dist,x,y,z
0,b'* alf Aur',05 16 41.3587,+45 59 52.769,9,9,4.7500,2.4700,90,A,O,...,NaN,NaN,paral,2010A&ARv..18...67T,162.588476,4.566431,13.0000,-12.364967,3.877677,-1.034994
1,b'4U 1642-45',16 45 47.7,-45 36 40,5,5,NaN,NaN,0,D,,...,NaN,NaN,,2002A&A...391..923G,339.588139,-0.079415,11000.0000,10309.298040,-3836.423003,15.246652
2,b'V* V1101 Sco',17 05 44.4915,-36 25 23.049,14,14,0.1502,0.1189,90,A,O,...,NaN,NaN,,2002A&A...391..923G,349.103609,2.748420,9200.0000,9023.737851,-1737.108517,-441.145329
3,b'NAME Sgr A*',17 45 40.0359,-29 00 28.169,9,9,2.6500,1.4200,0,B,R,...,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN
4,b'[KRL2007b] 312',18 17 43.537,-33 01 07.80,7,7,500.0000,500.0000,90,C,R,...,-1.5000,1.5000,,2006ATel..791....1S,359.817224,-7.995567,2500.0000,2475.684489,-7.897584,347.741199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,b'HD 191612',20 09 28.6111,+35 44 01.287,14,14,0.0250,0.0327,90,A,O,...,-161.0218,161.0218,paral,2018yCat.1345....0G,72.991665,1.434389,2126.7546,621.903802,2033.097778,-53.237339
1153,b'EM* AS 431',20 36 43.6345,+40 21 07.453,14,14,0.2438,0.2996,90,A,O,...,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN
1154,b'QSO B1725-142',17 28 19.7893,-14 15 55.855,14,14,0.0352,0.0305,90,A,O,...,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN
1155,b'* eta Car',10 45 03.5455,-59 41 03.951,11,11,11.0000,10.0000,90,B,O,...,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN


In [321]:
ScatterSources.to_csv('Show Dust and Sources_scattersources.csv')

## At this point we have produced the list of X-ray sources along with their coordinates.

The results was saved to a csv, so in the future we can pick up processing from here and we don't have to requery the 3D positions of all the X-ray sources.

In [328]:
ScatterSources = pd.read_csv('Show Dust and Sources_scattersources.csv')

In [370]:
# Compute the special coordinate of the Orion nebula.  Makes it easy to get our bearings.
c = SkyCoord(209.0137*u.deg, -19.3816*u.deg, 500*u.pc, frame='galactic')
x,y,z = Spherical2Cartesian(r=c.distance.pc, phi=c.l.deg, theta=c.b.deg)
Orionx = np.array((x,))
Oriony = np.array((y,))
Orionz = np.array((z,))

# Compute the special coordinate of the Cyg X-1.  Makes it easy to get our bearings.
c = SkyCoord(71.33499*u.deg, 3.0668*u.deg, 2370*u.pc, frame='galactic')
x,y,z = Spherical2Cartesian(r=c.distance.pc, phi=c.l.deg, theta=c.b.deg)
CygX1x = np.array((x,))
CygX1y = np.array((y,))
CygX1z = np.array((z,))

In [373]:
fig = ipv.figure()
#ipv.volshow(drho1kpc, level=[0.05], level_width=0.3, data_min=0.01, data_max=0.2, opacity=0.15)
NearScatterSources = ScatterSources.query('dist<=10000')
ipv.scatter(x=NearScatterSources['x'].to_numpy(), 
            y=NearScatterSources['y'].to_numpy(), 
            z=NearScatterSources['z'].to_numpy(), 
            color='blue',
            size=2)
ipv.scatter(x=Orionx, 
            y=Oriony, 
            z=Orionz, 
            color='green',
            size=2)
ipv.scatter(x=CygX1x, 
            y=CygX1y, 
            z=CygX1z, 
            color='cyan',
            size=3)
ipv.volshow(drho10kpc, extent=[[-10000,10000], [-10000,10000], [-10000,10000]], 
            level=[0.05], level_width=0.3, data_min=0.00, data_max=0.2, opacity=[0.5,0.5,0.5], memorder='F')
ipv.show()
ipv.save('10kpc Dust with X-ray Sources.html')

In [377]:
fig = ipv.figure()
#ipv.volshow(drho1kpc, level=[0.05], level_width=0.3, data_min=0.01, data_max=0.2, opacity=0.15)
NearScatterSources = ScatterSources.query('dist<=1000')
ipv.scatter(x=NearScatterSources['x'].to_numpy(), 
            y=NearScatterSources['y'].to_numpy(), 
            z=NearScatterSources['z'].to_numpy(), 
            color='blue',
            size=2)
ipv.scatter(x=Orionx, 
            y=Oriony, 
            z=Orionz, 
            color='green',
            size=2)
ipv.volshow(drho1kpc, extent=[[-1000,1000], [-1000,1000], [-1000,1000]], 
            level=[0.05], level_width=0.3, data_min=0.00, data_max=0.2, opacity=[0.5,0.5,0.5], memorder='F')
ipv.show()
ipv.save('1kpc Dust with X-ray Sources.html')